
## Information-Seeking Agent (Mini Research Assistant)

**Goal:**  
Create an agent that searches the web and summarizes its findings.  
*(Shows: retrieval + summarization pipeline)*



In [5]:
# Information-Seeking Agent — Clean, Formatted Output Version
# ---------------------------------------------------------------
# pip install wikipedia transformers torch --quiet

import wikipedia
from transformers import pipeline
import textwrap

# Initialize summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Step 1: Define the topic
topic = "Renewable energy in Germany"

print(" Topic Selected:", topic)
print("──────────────────────────────────────────────")

# Step 2: Retrieve information from Wikipedia
raw_text = wikipedia.summary(topic, sentences=10)

print(" Retrieved Context:")
print(textwrap.fill(raw_text[:700], width=100))
print("...")

# Step 3: Summarize the retrieved content
summary = summarizer(raw_text, max_length=120, min_length=40, do_sample=False)[0]["summary_text"]

print("\n Agent Summary:")
print("──────────────────────────────────────────────")
print(textwrap.fill(summary, width=100))


Device set to use mps:0


 Topic Selected: Renewable energy in Germany
──────────────────────────────────────────────
 Retrieved Context:
Renewable energy in Germany is mainly based on wind, solar and biomass.  In 2024, 59,0 % (254,9 TWh
of 431,7 TWh) of the electricity produced in Germany came from renewable Energies:  31,87 %   Wind
(Onshore 25,92 % + offshore 5,95 %), Photovoltaics 14,66, Biomass 8,33 %, hydropower 3,97 %. Germany
had the world's largest photovoltaic installed capacity until 2014, and as of 2023 it had over 82
GW. In 2021, it was the world's third country by installed total wind power capacity, 64 GW in 2021
and second for offshore wind, with over 7 GW. In 2009, Germany was called "the world's first major
renewable energy economy". The share of renewable energy in electricity production increased from
3.5%
...

 Agent Summary:
──────────────────────────────────────────────
In 2024, 59,0 % (254,9 TWh of 431,7 TWh) of the electricity produced in Germany came from renewable
Energies. Renewable 

## Exercise 2 — News Analyst Agent

**Goal:**  
Retrieve current news and analyze the *sentiment* of what it finds.  
*(Shows: retrieval + evaluation pipeline)*


In [10]:
# pip install ddgs transformers torch --quiet

from ddgs import DDGS
from transformers import pipeline
import textwrap

# Step 1: Initialize sentiment pipeline
sentiment = pipeline(
    "sentiment-analysis",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english"
)

# Step 2: Define query
query = "latest AI regulation news 2024 site:bbc.com OR site:reuters.com"

print(" AGENT QUERY:")
print("──────────────────────────────────────────────")
print(textwrap.fill(query, width=90))
print("──────────────────────────────────────────────\n")

# Step 3: Search recent news using DuckDuckGo / DDGS
with DDGS() as ddgs:
    results = [r["title"] + ". " + r["body"] for r in ddgs.text(query, max_results=3)]

# Combine results into context
context = " ".join(results)

# Display retrieved headlines
print("RETRIEVED HEADLINES:")
print("──────────────────────────────────────────────")
if results:
    for i, res in enumerate(results, 1):
        print(f"{i}. {textwrap.fill(res[:150], width=90)}\n")
else:
    print(" No results found. Try another query or site filter.\n")

# Step 4: Analyze sentiment of combined text
analysis = sentiment(context[:512])
print(" SENTIMENT SUMMARY:")
print("──────────────────────────────────────────────")
for a in analysis:
    label = a["label"]
    score = a["score"]
    print(f"{label:<10} → confidence {score:.2f}")


Device set to use mps:0


 AGENT QUERY:
──────────────────────────────────────────────
latest AI regulation news 2024 site:bbc.com OR site:reuters.com
──────────────────────────────────────────────

RETRIEVED HEADLINES:
──────────────────────────────────────────────
1. EU sticks with timeline for AI rules | Reuters. The European Union's landmark rules on
artificial intelligence will be rolled out according to the leg

2. AI News | Latest Headlines and Developments | Reuters. Explore the latest artificial
intelligence news with Reuters - from AI breakthroughs and techno

3. US senators call for Meta probe after Reuters report on its AI policies. Two Republican
U.S. senators called for a congressional investigation into Me

 SENTIMENT SUMMARY:
──────────────────────────────────────────────
NEGATIVE   → confidence 0.95


# Customer Support Agent

**Goal:**  
Detect user intent and decide on an action.  
*(Shows: perception → decision → action loop)*

In [11]:

# pip install transformers torch --quiet

from transformers import pipeline
import textwrap

# Step 1 – Initialize zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Step 2 – Define the incoming message
customer_message = "My package hasn't arrived even after 10 days and no one is replying to my emails!"

print("CUSTOMER MESSAGE:")
print("──────────────────────────────────────────────")
print(textwrap.fill(customer_message, width=90))
print("──────────────────────────────────────────────\n")

# Step 3 – Classify intent
candidate_labels = ["complaint", "refund request", "greeting", "product inquiry", "technical issue"]
prediction = classifier(customer_message, candidate_labels=candidate_labels)
intent = prediction["labels"][0]
confidence = prediction["scores"][0]

print("DETECTED INTENT:")
print("──────────────────────────────────────────────")
print(f"{intent.upper()}  →  confidence {confidence:.2f}\n")

# Step 4 – Decide an appropriate action
print("AGENT DECISION:")
print("──────────────────────────────────────────────")
if intent in ["complaint", "refund request"]:
    response = (
        "We’re sorry for the delay. I’ll create a priority support ticket and "
        "update you within 24 hours. Thank you for your patience!"
    )
elif intent == "technical issue":
    response = (
        "Please describe the issue in detail and we’ll connect you with a "
        "technical specialist right away."
    )
elif intent == "greeting":
    response = "Hello! How can I assist you today?"
else:
    response = (
        "I’d be happy to provide product information or track your order. "
        "Could you share your order ID?"
    )

print(textwrap.fill(response, width=90))


Device set to use mps:0


CUSTOMER MESSAGE:
──────────────────────────────────────────────
My package hasn't arrived even after 10 days and no one is replying to my emails!
──────────────────────────────────────────────

DETECTED INTENT:
──────────────────────────────────────────────
COMPLAINT  →  confidence 0.80

AGENT DECISION:
──────────────────────────────────────────────
We’re sorry for the delay. I’ll create a priority support ticket and update you within 24
hours. Thank you for your patience!


# Design Your Own Mini Agent
**Goal:**  
Apply what you learned about agents (reasoning, tool use, reflection) to build your own small task-oriented agent.  
This could be:
- a **news sentiment agent**,  
- a **product recommendation agent**, or  
- a **fact-checking agent**.

**You will:**
1. Define your own topic or problem.  
2. Retrieve text from the web (using DDGS or Wikipedia).  
3. Analyze or reason about that text using an LLM pipeline.  
4. Print your findings in a readable, agentic format.  


In [12]:
# pip install ddgs transformers torch wikipedia --quiet

from ddgs import DDGS
from transformers import pipeline
import textwrap, wikipedia

# Choose your own topic
topic = input("Enter your agent’s topic or task (e.g., 'AI in education 2024'): ")

#  Choose the agent type
print("\nChoose agent type: (1) Research summarizer  (2) Sentiment analyzer  (3) Fact-checker")
choice = input("Enter 1, 2, or 3: ")

print("\n AGENT QUERY:")
print("──────────────────────────────────────────────")
query = f"{topic} site:bbc.com OR site:reuters.com"
print(textwrap.fill(query, width=90))
print("──────────────────────────────────────────────\n")

#  Retrieve text (students can modify)
context = ""
with DDGS() as ddgs:
    results = [r["title"] + ". " + r["body"] for r in ddgs.text(query, max_results=3)]
    context = " ".join(results)

if not context:
    print("No search results found. Trying Wikipedia instead.")
    try:
        context = wikipedia.summary(topic, sentences=5)
    except:
        print("⚠️ Wikipedia also has no entry for this topic.")
        context = ""

print("RETRIEVED CONTEXT:")
print("──────────────────────────────────────────────")
print(textwrap.fill(context[:500], width=90))
print("...")

#  Based on agent type — students fill in logic
if choice == "1":
    print("\n TASK: Summarize the retrieved text\n")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(context[:2000], max_length=120, min_length=40, do_sample=False)[0]["summary_text"]
    print("SUMMARY RESULT:")
    print("──────────────────────────────────────────────")
    print(textwrap.fill(summary, width=90))

elif choice == "2":
    print("\n TASK: Sentiment analysis of retrieved text\n")
    sentiment = pipeline("sentiment-analysis")
    analysis = sentiment(context[:512])
    print("SENTIMENT RESULT:")
    print("──────────────────────────────────────────────")
    for a in analysis:
        label = a["label"]
        score = a["score"]
        print(f" {label:<10} → confidence {score:.2f}")

elif choice == "3":
    print("\nTASK: Fact-check the main claim (students implement their logic)\n")
    print("# Optional: use a question-answering pipeline here")
    print("# Example:")
    print("# qa = pipeline('question-answering', model='deepset/roberta-base-squad2')")
    print("# claim = input('Enter a claim to verify: ')")
    print("# answer = qa(question=claim, context=context)")
    print("# print(' Agent conclusion:', answer['answer'])")
else:
    print(" Invalid choice. Please rerun and select 1, 2, or 3.")



Choose agent type: (1) Research summarizer  (2) Sentiment analyzer  (3) Fact-checker

 AGENT QUERY:
──────────────────────────────────────────────
summary site:bbc.com OR site:reuters.com
──────────────────────────────────────────────



/var/folders/36/c6m7q9gx4njbdgdy1y3mj9gm0000gn/T/ipykernel_24537/2079546550.py:24: ResourceWarning: unclosed <ssl.SSLSocket fd=96, family=2, type=1, proto=0, laddr=('192.168.178.96', 59516), raddr=('40.114.177.156', 443)>
  with DDGS() as ddgs:


RETRIEVED CONTEXT:
──────────────────────────────────────────────
Reuters | Breaking International News & Views. Find latest news from every corner of the
globe at Reuters.com , your online source for breaking international news coverage. Global
Market Headlines | Breaking Stock Market News | Reuters. Find the latest stock market news
from every corner of the globe at Reuters.com , your online source for breaking
international market and finance news US economy at risk of wobble as lower-income
consumers get squeezed. The U.S. consumer's durability as a prop f
...
 Invalid choice. Please rerun and select 1, 2, or 3.
